In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import regex as re
import ipywidgets as widgets
from IPython.display import display
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import random


scaler = StandardScaler()

moviedf = pd.read_csv("rotten_tomatoes_movies.csv")
criticsdf = pd.read_csv("rotten_tomatoes_critic_reviews.csv")

In [2]:
def clean_title(a):
    return re.sub("[^a-zA-Z0-9 ]","",a)

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = moviedf.iloc[indices].iloc[::-1]
    
    return results

moviedf['genres'] = moviedf['genres'].fillna('Unknown')
print(moviedf['genres'].isna().sum())

0


In [3]:
moviedf["Clean Titles"] = moviedf["movie_title"].apply(clean_title)
moviedf['genres'] = moviedf['genres'].str.split(',')
moviedf['genres'] = moviedf['genres'].apply(lambda x: [genre.replace(' ', '') for genre in x])
moviedf['genres'] = moviedf['genres'].apply(lambda x: '-'.join(x))
moviedf['genres'] = moviedf['genres'].apply(lambda x: x.split('-'))
all_genres = set([genre for genres_list in moviedf['genres'] for genre in genres_list])
dummies = pd.get_dummies(moviedf['genres'].apply(pd.Series).stack(), columns=all_genres).sum(level=0)
moviedf = pd.concat([moviedf, dummies], axis=1)

dummies

C:\Users\Sarab\AppData\Local\Temp\ipykernel_17292\2876693538.py:7: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(moviedf['genres'].apply(pd.Series).stack(), columns=all_genres).sum(level=0)


,Action&Adventure,Animation,Anime&Manga,ArtHouse&International,Classics,Comedy,CultMovies,Documentary,Drama,Faith&Spirituality,...,Kids&Family,Musical&PerformingArts,Mystery&Suspense,Romance,ScienceFiction&Fantasy,SpecialInterest,Sports&Fitness,Television,Unknown,Western
0,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
17708,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17709,1,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
17710,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
moviedf

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,Kids&Family,Musical&PerformingArts,Mystery&Suspense,Romance,ScienceFiction&Fantasy,SpecialInterest,Sports&Fitness,Television,Unknown,Western
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"[Action&Adventure, Comedy, Drama, ScienceFicti...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,0,0,0,0,1,0,0,0,0,0
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,[Comedy],Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,0,0,0,0,0,0,0,0,0,0
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"[Comedy, Romance]",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,0,0,0,1,0,0,0,0,0,0
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"[Classics, Drama]",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,0,0,0,0,0,0,0,0,0,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"[Action&Adventure, Drama, Kids&Family]",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,m/zoot_suit,Zoot Suit,Mexican-American gangster Henry Reyna (Daniel ...,NaN,R,"[Drama, Musical&PerformingArts]",Luis Valdez,Luis Valdez,"Daniel Valdez, Edward James Olmos, Charles Aid...",1981-10-02,...,0,1,0,0,0,0,0,0,0,0
17708,m/zootopia,Zootopia,From the largest elephant to the smallest shre...,The brilliantly well-rounded Zootopia offers a...,PG,"[Action&Adventure, Animation, Comedy]","Byron Howard, Rich Moore, Jared Bush","Jared Bush, Phil Johnston","J.K. Simmons, Kristen Bell, Octavia Spencer, A...",2016-03-04,...,0,0,0,0,0,0,0,0,0,0
17709,m/zorba_the_greek,Zorba the Greek,Traveling to inspect an abandoned mine his fat...,NaN,NR,"[Action&Adventure, ArtHouse&International, Cla...",NaN,NaN,"Anthony Quinn, Alan Bates, Irene Papas, Lila K...",1964-12-17,...,0,0,0,0,0,0,0,0,0,0
17710,m/zulu,Zulu,"In 1879, the Zulu nation hands colonial Britis...",Zulu patiently establishes a cast of colorful ...,PG,"[Classics, Drama]","Cy Endfield, Cyril Endfield","Cy Endfield, John Prebble","Stanley Baker, Jack Hawkins, Ulla Jacobsson, J...",1964-06-17,...,0,0,0,0,0,0,0,0,0,0


In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf = vectorizer.fit_transform(moviedf["Clean Titles"])

In [6]:
equivalents = {
    "A+": 5,
    "A": 4.7058823528,
    "A-": 4.41176470575,
    "B+": 4.1176470587,
    "B": 3.82352941165,
    "B-": 3.5294117646,
    "C+": 3.23529411755,
    "C": 2.9411764705,
    "C-": 2.64705882345,
    "D+": 2.3529411764,
    "D": 2.05882352935,
    "D-": 1.7647058823,
    "E+": 1.47058823525,
    "E": 1.1764705882,
    "E-": 0.88235294115,
    "F+": 0.5882352941,
    "F": 0
    }

def standardize(rev):
    if "/" in str(rev):
        return float(rev[:rev.index("/")])
    else:
        score = equivalents.get(rev)
        if score is not None:
            return score
        else:
            return None
        
def standardize2(rev):
    if rev > 5:
        return rev / 20

criticsdf.dropna(subset="review_score",inplace=True)
criticsdf["review_score"] = criticsdf["review_score"].apply(standardize)
criticsdf.drop("review_content",axis=1,inplace=True)
criticsdf.drop("review_date",axis=1,inplace=True)
criticsdf.reset_index(inplace=True)

In [7]:
#Swap columns around
columns_titles = ["critic_name","top_critic","publisher_name","rotten_tomatoes_link","review_score","review_type"]
criticsdf=criticsdf.reindex(columns=columns_titles)
criticsdf.dropna(axis=0,inplace=True)
criticsdf['review_score'] = criticsdf['review_score'].round().astype(int)
criticsdf

,critic_name,top_critic,publisher_name,rotten_tomatoes_link,review_score,review_type
0,Ben McEachen,False,Sunday Mail (Australia),m/0814255,4,Fresh
1,Nick Schager,False,Slant Magazine,m/0814255,1,Rotten
2,Bill Goodykoontz,True,Arizona Republic,m/0814255,4,Fresh
3,Jordan Hoffman,False,UGO,m/0814255,4,Fresh
4,Jim Schembri,True,The Age (Australia),m/0814255,3,Fresh
...,...,...,...,...,...,...
824076,Chuck O'Leary,False,Fantastica Daily,m/zulu_dawn,2,Rotten
824077,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",m/zulu_dawn,4,Fresh
824078,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,m/zulu_dawn,4,Fresh
824079,Christopher Lloyd,False,Sarasota Herald-Tribune,m/zulu_dawn,4,Rotten


In [8]:
mov="m/man_who_would_be_king"
moviehaters = criticsdf[(criticsdf['rotten_tomatoes_link'] == mov) &
                            ((criticsdf['review_score'] == 1) | (criticsdf['review_score'] == 2))]
#movieboys = criticdummies[(criticdummies['review_type'] == "Fresh")]
crits = criticsdf["critic_name"].values.tolist()
moviehaters

,critic_name,top_critic,publisher_name,rotten_tomatoes_link,review_score,review_type


In [9]:
chosenones = criticsdf.loc[(criticsdf['critic_name'].isin(crits))]
chosenones["review_score"] = chosenones["review_score"].apply(standardize2)
chosenones

,critic_name,top_critic,publisher_name,rotten_tomatoes_link,review_score,review_type
0,Ben McEachen,False,Sunday Mail (Australia),m/0814255,NaN,Fresh
1,Nick Schager,False,Slant Magazine,m/0814255,NaN,Rotten
2,Bill Goodykoontz,True,Arizona Republic,m/0814255,NaN,Fresh
3,Jordan Hoffman,False,UGO,m/0814255,NaN,Fresh
4,Jim Schembri,True,The Age (Australia),m/0814255,NaN,Fresh
...,...,...,...,...,...,...
824076,Chuck O'Leary,False,Fantastica Daily,m/zulu_dawn,NaN,Rotten
824077,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",m/zulu_dawn,NaN,Fresh
824078,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,m/zulu_dawn,NaN,Fresh
824079,Christopher Lloyd,False,Sarasota Herald-Tribune,m/zulu_dawn,NaN,Rotten


In [10]:
average_scores = chosenones.groupby('rotten_tomatoes_link')['review_score'].mean()
num_votes = chosenones.groupby('rotten_tomatoes_link')['review_type'].count()

In [17]:
score_weight = 1.2
votes_weight = -0.1

In [80]:
chosenflicks = pd.DataFrame({'rotten_tomatoes_link': chosenones['rotten_tomatoes_link'].unique()})
chosenflicks['AverageScore'] = chosenflicks['rotten_tomatoes_link'].map(average_scores)
chosenflicks['NumVotes'] = chosenflicks['rotten_tomatoes_link'].map(num_votes)
chosenflicks['ObjectiveScore'] = chosenflicks.apply(lambda row: (score_weight * row['AverageScore']) + (votes_weight * row['NumVotes']) + random.uniform(-0.6, 0.6), axis=1)
chosenflicks = chosenflicks.sort_values(['ObjectiveScore'], ascending=False)

chosenflicks

,rotten_tomatoes_link,AverageScore,NumVotes,ObjectiveScore
5876,m/eddie_murphy_delirious,4.45,4,5.425137
13422,m/small_town_gay_bar,4.30,4,5.164063
108,m/10008661-randy_and_the_mob,4.45,7,4.960747
16183,m/three_of_hearts,4.60,9,4.945981
1242,m/1090492-war_at_home,4.10,2,4.932963
...,...,...,...,...
17663,m/zookeeper,NaN,109,NaN
17666,m/zoom_2006,NaN,56,NaN
17667,m/zoot_suit,NaN,5,NaN
17670,m/zulu,NaN,13,NaN


# Stuff over here is unlikely to be used

In [13]:
X=chosenones.loc[:, chosenones.columns != "review_score"]
y=chosenones["review_score"]

In [14]:
pivot = np.round(pd.pivot_table(chosenones, values='review_score', 
                                index='', 
                                columns='fuel-type', 
                                aggfunc=np.mean),2)
pivot

KeyError: ''

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=55, train_size = 0.75)
print("The length of the initial dataset is :", len(X))
print("The length of the train dataset is   :", len(X_train))
print("The length of the test dataset is    :", len(X_test))

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
print("\nScore for the Train dataset :", model.score(X_train, y_train))
print("Score for the Test dataset :", model.score(X_test, y_test))
#X_pca = pca.fit_transform(criticdummies.iloc[:170000])

In [ ]:
criticsdf["rotten_tomatoes_link"].value_counts()

In [ ]:
#In progress search engine. Streamlit has its own native search bar though, so most likely this code will never
#be needed and therefore will remain abandoned.


movie_input = widgets.Text(
    value='The Big Lebowski',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

In [ ]:
criticdummies = pd.get_dummies(criticsdf,columns=["critic_name"])
#criticdummies = criticdummies.select_dtypes(include = "number")   #The sheer number of dummy columns makes this kinda silly
criticdummies.dropna(axis=0,inplace=True)
criticdummies['review_score'] = criticdummies['review_score'].round().astype(int)
criticdummies
#criticdummies.drop(["review_score"], axis=1,inplace=True)
#criticdummies2 = df.loc[:, df.columns != "review_score"]

In [ ]:
movieid = "m/falling_down"
similar_users = criticsdf[(criticsdf["rotten_tomatoes_link"]==movieid) | (criticsdf["review_score"] > 4)]["critic_name"].unique()
similar_users

In [ ]:
criticsdf

In [ ]:
similar_userecs = criticsdf[(criticsdf["critic_name"].isin(similar_users)) & (criticsdf["review_score"] > 4)]["rotten_tomatoes_link"]
similar_userecs = similar_userecs.value_counts() / len(similar_users)
similar_userecs

In [ ]:
all_users = criticsdf[(criticsdf["rotten_tomatoes_link"].isin(similar_userecs.index)) & (criticsdf["review_score"] > 4)]
all_user_recs = all_users["rotten_tomatoes_link"].value_counts() / len(all_users["critic_name"].unique())

all_user_recs

In [ ]:
rec_percentages = pd.concat([similar_userecs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [ ]:
rec_percentages

In [ ]:
similar_userecs.head(5)
#similar_userecs = similar_userecs[similar_userecs > 0.001]